In [1]:
import http.client
import json
import pandas as pd
from pandas import DataFrame, Series
import logging
import logging.handlers  

In [2]:
# 设置日志
LOG_FILE = 'word_explain.log'  
  
handler = logging.handlers.RotatingFileHandler(LOG_FILE, maxBytes = 1024*1024) # 实例化handler    
fmt = '%(asctime)s - %(message)s'  
  
formatter = logging.Formatter(fmt)   # 实例化formatter   
handler.setFormatter(formatter)      # 为handler添加formatter   
  
logger = logging.getLogger('getExplainFromYoudao')    # 日志文件名
logger.addHandler(handler)           # 为logger添加handler   
logger.setLevel(logging.DEBUG)  

In [3]:
# 使用有道词典API查询一个单词的含义
def getExplain(word, keyFrom, key):
    # 有道词典API形如：
    # http://fanyi.youdao.com/openapi.do?keyfrom=batchtranslate&key=1679006548&type=data&doctype=json&version=1.1&q=process
    youdao_api_url='/openapi.do?keyfrom='+keyFrom + '&key=' + key + '&type=data&doctype=json&version=1.1&q='
    try:
        conn.request("GET", youdao_api_url + word)
    except:
        conn = http.client.HTTPConnection("fanyi.youdao.com", 80)
        conn.request("GET", youdao_api_url + word)
    r1 = conn.getresponse()
    # 从返回的JSON字符串中提取出单词释义
    data1 = r1.read()
    jsonStr = data1.decode('utf-8')
    jsonObj = json.loads(jsonStr)
    if('basic' in jsonObj):
        explainObj = jsonObj['basic']['explains']
        firstExplainStr = '  '.join(explainObj)
        return firstExplainStr
    logger.warning('explain not found, word = ' + word)
    return 'NA' # 没有查到释义


In [4]:
# 读取单词到DataFrame
wordsDataFrame = pd.read_table('/Users/maoshuai/maos/tmp/words.txt',header=None, names=('num','word'))
logger.info('word loaded.')

In [5]:
startRow=1398 # 翻译的起始行，上次断点的地方
total = len(wordsDataFrame) # 所有单词数量
leftRows = wordsDataFrame[startRow:]
count = startRow # 记录正在翻译的行，主要为了记录断点
# youdaoDictKeyFrom='batchtranslate'
# youdaoDictKey='1679006548'

youdaoDictKeyFrom='TEST1234556423'
youdaoDictKey='1161529587'

while count < total:
    word = wordsDataFrame.ix[count, 'word']
    try:
        explain = getExplain(word, youdaoDictKeyFrom, youdaoDictKey)
        wordsDataFrame.ix[count, 'explain'] = explain
        logger.info('explain: ' + str(count) + "/" + str(total) + ": " + word)
        count += 1
    except:
        logger.error("break row: " + str(count) + ", word: " + str(word))
        break

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:19: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead


In [6]:
# 翻译结果，写到CSV文件
if(startRow < count):
    explainFile = '/Users/maoshuai/maos/tmp/words_with_explain_' + str(startRow) + '_' + str(count) + '.csv'
    wordsDataFrame[startRow:count].to_csv(explainFile)
    logger.info('save to: ' + explainFile)
else:
    logger.info('no file to save')
